<a href="https://colab.research.google.com/github/hsreedharanDL/Human-Activity-Recognition-with-Tensorflow/blob/main/humanActivityRecognitionlstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import RobustScaler
keras = tf.keras

In [18]:
!gdown --id 152sWECukjvLerrVG2NUO8gtMFg83RKCF --output WISDM_ar_latest.tar.gz
!tar -xvf WISDM_ar_latest.tar.gz

Downloading...
From: https://drive.google.com/uc?id=152sWECukjvLerrVG2NUO8gtMFg83RKCF
To: /content/WISDM_ar_latest.tar.gz
11.4MB [00:00, 99.6MB/s]
WISDM_ar_v1.1/
WISDM_ar_v1.1/readme.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_raw_about.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_transformed.arff
WISDM_ar_v1.1/WISDM_ar_v1.1_trans_about.txt


In [19]:
column_names = [
  'user_id',
  'activity',
  'timestamp',
  'x_axis',
  'y_axis',
  'z_axis'
]
df = pd.read_csv(
  'WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt',
  header=None,
  names=column_names
)
df.z_axis.replace(regex=True, inplace=True, to_replace=r';', value=r'')
df['z_axis'] = df.z_axis.astype(np.float64)
df.dropna(axis=0, how='any', inplace=True)
df.shape

(1098203, 6)

In [20]:
df_train = df[df['user_id'] <= 30]
df_test = df[df['user_id'] > 30]

In [21]:
scale_columns = ['x_axis', 'y_axis', 'z_axis']
scaler = RobustScaler()
scaler = scaler.fit(df_train[scale_columns])
df_train.loc[:, scale_columns] = scaler.transform(
  df_train[scale_columns].to_numpy()
)
df_test.loc[:, scale_columns] = scaler.transform(
  df_test[scale_columns].to_numpy()
)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [22]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [23]:
TIME_STEPS = 200
STEP = 40
X_train, y_train = create_dataset(
    df_train[['x_axis', 'y_axis', 'z_axis']],
    df_train.activity,
    TIME_STEPS,
    STEP
)
X_test, y_test = create_dataset(
    df_test[['x_axis', 'y_axis', 'z_axis']],
    df_test.activity,
    TIME_STEPS,
    STEP
)

In [24]:
print(X_train.shape, y_train.shape)

(22454, 200, 3) (22454, 1)


In [25]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc = enc.fit(y_train)
y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

In [26]:
from tensorflow import keras
from keras.layers import Dense
from tensorflow.keras import layers

In [27]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=128,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

In [28]:
history=model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1,
    shuffle=True)

Epoch 1/10
632/632 [==============================] - 188s 291ms/step - loss: 1.1820 - acc: 0.5943 - val_loss: 1.1595 - val_acc: 0.5766
Epoch 2/10
632/632 [==============================] - 180s 285ms/step - loss: 0.9284 - acc: 0.6950 - val_loss: 1.1846 - val_acc: 0.6465
Epoch 3/10
632/632 [==============================] - 181s 286ms/step - loss: 0.7879 - acc: 0.7293 - val_loss: 0.6701 - val_acc: 0.7654
Epoch 4/10
632/632 [==============================] - 180s 284ms/step - loss: 0.7551 - acc: 0.7360 - val_loss: 0.6431 - val_acc: 0.8344
Epoch 5/10
632/632 [==============================] - 181s 286ms/step - loss: 0.5688 - acc: 0.7953 - val_loss: 0.4860 - val_acc: 0.8495
Epoch 6/10
632/632 [==============================] - 182s 287ms/step - loss: 0.7392 - acc: 0.7457 - val_loss: 0.7916 - val_acc: 0.7605
Epoch 7/10
632/632 [==============================] - 181s 286ms/step - loss: 0.8874 - acc: 0.6787 - val_loss: 0.7810 - val_acc: 0.6852
Epoch 8/10
632/632 [============================

In [29]:
model.evaluate(X_test, y_test)

156/156 [==============================] - 11s 68ms/step - loss: 0.6191 - acc: 0.8123


[0.6191031336784363, 0.8122996687889099]

In [30]:
y_pred = model.predict(X_test)